Author: Kevin ALBERT

Created: Feb 2021

In [1]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from IPython.display import Javascript
from fuzzywuzzy import process, fuzz

In [2]:
import platform
import psutil
import os

In [3]:
conda_version = ! conda -V
print(f"conda : {conda_version[0].split()[1]}")
pip_version = ! pip -V
print(f"pip   : {pip_version[0].split()[1]}")
python_version = ! python -V
print(f"python: {python_version[0].split()[1]}")
pandas_version = ! pip list |grep -i pandas
print(f"pandas: {pandas_version[0].split()[1]}")
py2neo_version = ! pip list |grep -i py2neo
print(f"py2neo: {py2neo_version[0].split()[1]}")

conda : 4.9.2
pip   : 21.0.1
python: 3.8.6
pandas: 1.2.1
py2neo: 4.2.0


In [4]:
server  = "168.63.44.210"
port    = "7687"
user    = "neo4j"
passw   = "digityser"
db_name = "neo4j"        # default name (v4.x)

In [5]:
# load graph connection instance
graph = Graph(host=server, auth=(user, passw), name=db_name, encrypted=False)

## prepare data
We may be trying to do it for all 3 dataset groups !

In [7]:
# import the datasets
synthetic_df = pd.read_parquet("../../data/silver/synthetic_data_processed.parquet")
entities_df = pd.read_parquet("../../data/bronze/entities_What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction.parquet")
sentiment_df = pd.read_parquet("../../data/bronze/sentiment_What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction.parquet")
keyphrase_df = pd.read_parquet("../../data/bronze/key_phrases_What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction.parquet")

In [8]:
# creating unique patient ID
synthetic_df["id"] = synthetic_df.index

In [9]:
# replace nan
synthetic_df['What_Felt_Best_During_This_Last_Month'] = synthetic_df['What_Felt_Best_During_This_Last_Month'].fillna('No Feeling')
synthetic_df['What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction'] = synthetic_df['What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction'].fillna('No Satisfaction')
synthetic_df['Any_Additional_Suggestion_To_Improve'] = synthetic_df['Any_Additional_Suggestion_To_Improve'].fillna('No Improvement')

In [10]:
synthetic_df

,email,full_name,how_was_your_last_month_in_your_assignment,How_Was_Your_Last_Month_Within_Your_Department,How_Was_Your_Last_Month_With_Us,What_Felt_Best_During_This_Last_Month,What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction,Any_Additional_Suggestion_To_Improve,any_additional_comments,I_Would_Like_To_Get_Called_By,I_Would_Like_To_Provide_A_Copy_Of_My_Answers_To,id
0,Lorem.ipsum@congueelit.ca,Harrison,4,4,4,"collaborating closely with Damien, closing th...","keep the good vibes, the trust and open commun...",more sync between the different departments,NaN,NaN,NaN,0
1,elit.elit.fermentum@Crasinterdum.ca,Clinton,4,4,4,being promoted,maintain the good collaboration,keep up the good energy,NaN,NaN,NaN,1
2,semper@mi.com,Scott,3,3,2,team building we had last month,"more consideration for the employes, making th...",Don’t be friends during work hours.,NaN,NaN,NaN,2
3,tellus.non@velsapien.co.uk,Russell,1,1,1,not much for me,I don't feel that my skills and capacities are...,If a few managers took a chill pill.,NaN,NaN,NaN,3
4,vitae@nisinibhlacinia.ca,Kane,4,4,3,getting to work with you guys is a real pleasure,less micro management,I would like to talk to the managers just to s...,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...
95,erat.vel@egestasrhoncusProin.edu,Victor,4,4,2,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,95
96,In@vitaeorci.net,Gareth,2,2,1,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,96
97,elementum@necurnaet.ca,Avram,4,4,4,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,97
98,quis.pede.Suspendisse@risus.edu,Coby,4,4,4,No Feeling,No Satisfaction,No Improvement,NaN,NaN,NaN,98


In [11]:
# count missing values, sorted (high missing > 5%)
synthetic_df.isnull().apply(lambda x: x.sum() * 100 / len(synthetic_df)).round(1).sort_values(ascending=True)

email                                                                                 0.0
full_name                                                                             0.0
how_was_your_last_month_in_your_assignment                                            0.0
How_Was_Your_Last_Month_Within_Your_Department                                        0.0
How_Was_Your_Last_Month_With_Us                                                       0.0
What_Felt_Best_During_This_Last_Month                                                 0.0
What_Should_We_Do_To_Make_You_Feel_Better_And_Increase_Your_Overall_Satisfaction      0.0
Any_Additional_Suggestion_To_Improve                                                  0.0
id                                                                                    0.0
any_additional_comments                                                             100.0
I_Would_Like_To_Get_Called_By                                                       100.0
I_Would_Li

In [14]:
# save file to database /import
file_synthetic = "synthetic_data_processed.csv"
synthetic_df.to_csv("../../neo4j/import/"+file_synthetic, sep=',', index=False)

file_entities3 = "entities3.csv"
entities_df.to_csv("../../neo4j/import/"+file_entities3, sep=',', index=False)

file_sentiment3 = "sentiment3.csv"
sentiment_df.to_csv("../../neo4j/import/"+file_sentiment3, sep=',', index=False)

file_keyphrase3 = "keyphrases3.csv"
keyphrase_df.to_csv("../../neo4j/import/"+file_keyphrase3, sep=',', index=False)

## import data in DB

[link to arrows data modeling tool](https://arrows.app/#/import/json=eyJncmFwaCI6eyJzdHlsZSI6eyJub2RlLWNvbG9yIjoiIzRDOEVEQSIsImJvcmRlci13aWR0aCI6MCwiYm9yZGVyLWNvbG9yIjoiIzAwMDAwMCIsInJhZGl1cyI6NzUsIm5vZGUtcGFkZGluZyI6NSwib3V0c2lkZS1wb3NpdGlvbiI6ImF1dG8iLCJjYXB0aW9uLXBvc2l0aW9uIjoiaW5zaWRlIiwiY2FwdGlvbi1tYXgtd2lkdGgiOjIwMCwiY2FwdGlvbi1jb2xvciI6IiNmZmZmZmYiLCJjYXB0aW9uLWZvbnQtc2l6ZSI6MjAsImNhcHRpb24tZm9udC13ZWlnaHQiOiJub3JtYWwiLCJsYWJlbC1wb3NpdGlvbiI6Imluc2lkZSIsImxhYmVsLWNvbG9yIjoiIzAwMDAwMCIsImxhYmVsLWJhY2tncm91bmQtY29sb3IiOiIjZmZmZmZmIiwibGFiZWwtYm9yZGVyLWNvbG9yIjoiIzg0ODQ4NCIsImxhYmVsLWJvcmRlci13aWR0aCI6MywibGFiZWwtZm9udC1zaXplIjoyMCwibGFiZWwtcGFkZGluZyI6NSwibGFiZWwtbWFyZ2luIjo0LCJkaXJlY3Rpb25hbGl0eSI6ImRpcmVjdGVkIiwiZGV0YWlsLXBvc2l0aW9uIjoiYWJvdmUiLCJkZXRhaWwtb3JpZW50YXRpb24iOiJwYXJhbGxlbCIsImFycm93LXdpZHRoIjozLCJhcnJvdy1jb2xvciI6IiM4NDg0ODQiLCJtYXJnaW4tc3RhcnQiOjUsIm1hcmdpbi1lbmQiOjUsIm1hcmdpbi1wZWVyIjoyMCwiYXR0YWNobWVudC1zdGFydCI6Im5vcm1hbCIsImF0dGFjaG1lbnQtZW5kIjoibm9ybWFsIiwidHlwZS1jb2xvciI6IiM4NDg0ODQiLCJ0eXBlLWJhY2tncm91bmQtY29sb3IiOiIjZmZmZmZmIiwidHlwZS1ib3JkZXItY29sb3IiOiIjODQ4NDg0IiwidHlwZS1ib3JkZXItd2lkdGgiOjAsInR5cGUtZm9udC1zaXplIjoyMSwidHlwZS1wYWRkaW5nIjo1LCJwcm9wZXJ0eS1wb3NpdGlvbiI6Im91dHNpZGUiLCJwcm9wZXJ0eS1jb2xvciI6IiM4NDg0ODQiLCJwcm9wZXJ0eS1mb250LXNpemUiOjIwLCJwcm9wZXJ0eS1mb250LXdlaWdodCI6Im5vcm1hbCJ9LCJub2RlcyI6W3siaWQiOiJuMCIsInBvc2l0aW9uIjp7IngiOjAsInkiOjB9LCJjYXB0aW9uIjoiRW1wbG95ZWUiLCJzdHlsZSI6e30sImxhYmVscyI6W10sInByb3BlcnRpZXMiOnt9fV0sInJlbGF0aW9uc2hpcHMiOltdfSwiZGlhZ3JhbU5hbWUiOiJzZHdvcnhIUmhhY2thdGhvbjIwMjFfZGF0YU1vZGVsIn0=)

#### cognitive data

In [15]:
entities_df

,document,entities,entity_text,entity_category,entity_subcategory,entity_confidence_score
0,"keep the good vibes, the trust and open commun...","{'text': 'communication', 'category': 'Skill',...",communication,Skill,None,0.8
1,maintain the good collaboration,"{'text': 'collaboration', 'category': 'Skill',...",collaboration,Skill,None,0.8
2,"more consideration for the employes, making th...","{'text': 'management decision', 'category': 'S...",management decision,Skill,None,0.8
3,less micro management,"{'text': 'micro management', 'category': 'Skil...",micro management,Skill,None,0.8
4,valuate all staff the same way,"{'text': 'valuate', 'category': 'Skill', 'subc...",valuate,Skill,None,0.8
5,more transparency about the financial situatio...,"{'text': 'financial', 'category': 'Skill', 'su...",financial,Skill,None,0.8
6,making the employees in the heart of managemen...,"{'text': 'management', 'category': 'Skill', 's...",management,Skill,None,0.8
7,more openness and transparency,"{'text': 'openness', 'category': 'Skill', 'sub...",openness,Skill,None,0.8
8,more training,"{'text': 'training', 'category': 'Skill', 'sub...",training,Skill,None,0.8
9,offer more opportunities for continuous learning,"{'text': 'continuous learning', 'category': 'S...",continuous learning,Skill,None,0.8


In [16]:
# count missing values, sorted (high missing > 5%)
entities_df.isnull().apply(lambda x: x.sum() * 100 / len(entities_df)).round(1).sort_values(ascending=True)

document                   0.0
entities                   0.0
entity_text                0.0
entity_category            0.0
entity_subcategory         0.0
entity_confidence_score    0.0
dtype: float64

#### replace Suggestion Node to either Feeling, Satisfaction or Suggestion into the other 2 notebooks!!
here it must be Satisfaction

In [17]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'Event'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (event:Event {value:(line.entity_text)})
MERGE (event)-[:EVENT_IN]->(satisfaction)
"""
display(graph.run(query).stats())
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'Location'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (location:Location {value:(line.entity_text)})
MERGE (location)-[:LOCATION_IN]->(satisfaction)
"""
display(graph.run(query).stats())
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'Product'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (product:Product {value:(line.entity_text)})
MERGE (product)-[:PRODUCT_IN]->(satisfaction)
"""
display(graph.run(query).stats())
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'Skill'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (skill:Skill {value:(line.entity_text)})
MERGE (skill)-[:SKILL_IN]->(satisfaction)
"""
display(graph.run(query).stats())
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'Quantity'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (quantity:Quantity {value:(line.entity_text)})
MERGE (quantity)-[:QUANTITY_IN]->(satisfaction)
"""
display(graph.run(query).stats())
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_entities3+"""' AS line FIELDTERMINATOR ','
WITH line WHERE line.entity_category = 'PersonType'
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (persontype:PersonType {value:(line.entity_text)})
MERGE (persontype)-[:PERSONTYPE_IN]->(satisfaction)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 14
labels_removed: 0
nodes_created: 14
nodes_deleted: 0
properties_set: 14
relationships_created: 19
relationships_deleted: 0

constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

constraints_added: 0
constraints_removed: 0
contains_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

#### sentiment

In [22]:
sentiment_df

,document,sentiment
0,"keep the good vibes, the trust and open commun...",positive
1,maintain the good collaboration,positive
2,"more consideration for the employes, making th...",neutral
3,I don't feel that my skills and capacities are...,negative
4,less micro management,neutral
...,...,...
95,none,neutral
96,none,neutral
97,none,neutral
98,none,neutral


In [20]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file_sentiment3+"""' AS line FIELDTERMINATOR ','
MATCH (satisfaction:Satisfaction {description:(line.document)})
MERGE (sentiment:Sentiment {value:(line.sentiment)})
MERGE (sentiment)-[:SENTIMENT_IN]->(satisfaction)
"""
display(graph.run(query).stats())

constraints_added: 0
constraints_removed: 0
contains_updates: True
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 37
relationships_deleted: 0

## Querying

## reset DB

In [ ]:
# delete database and restart (wait ~2min)
import os
os.system(" cd ../.. && \
            sudo docker-compose down && \
            sudo rm -Rf neo4j/data/databases/neo4j && \
            sudo rm -Rf neo4j/data/transactions/neo4j && \
            sudo docker-compose up --build &")